In [1]:
import asyncio
import pandas as pd
from vector_search import VectorSearch, build_context_string
from vexa import VexaAPI
from core import generic_call, system_msg, user_msg
from pydantic_models import Summary
from prompts import Prompts

from core import BaseCall
from pydantic import BaseModel, Field
from typing import List, Optional

In [ ]:
analyzer = VectorSearch(gpu_device=3)
vexa = VexaAPI()
prompts = Prompts()
await vexa.get_user_info()
meetings = await vexa.get_meetings()
meetings = meetings[-100:]


User information retrieved successfully.
Request URL: http://127.0.0.1:8001/api/v1/calls/all
Request Params: {'token': '3ae04e20124d40babc5107e658c666b6'}


In [69]:

meeting_id = meetings[13]['id']

trasncription = await vexa.get_transcription(meeting_session_id=meeting_id, use_index=True)
df, formatted_input, start_datetime, speakers = trasncription


df

,index,speaker,speaker_id,content,html_content,html_content_short,keywords,timestamp,formatted_time,time_tuple,initials,chunk_number
0,0,Sergey Ryabenko,TBD,"привет я на телефоне это готовлю тут ребенку еду а я хотел те экран пока показать давай смогу увидеть вряд ли там код надо читать смогу тогда чуть попозже быть насколько Ну, я надеюсь, сейчас где-то давай минут через 15.","Привет! Я на телефоне, готовлю еду для ребенка. Хотел показать тебе экран, но вряд ли смогу, там код надо читать. Если смогу, тогда чуть попозже. Надеюсь, сейчас где-то через 15 минут.",,[],2024-09-18T17:37:35.184000Z,00:00,"(0, 0)",SR,0
1,1,,TBD,"Да, ну смотри, так просто, мне просто интересно, хотел тебя познакомить.","Да, ну смотри, так просто. Мне просто интересно, хотел тебя познакомить.",,[],2024-09-18T17:37:42.624000Z,00:07,"(0, 7)",NaN,0
2,2,Dmitriy Grankin,TBD,показать и понять что тут вообще происходит вот я ему дал картинку интерфейса такой что картинку интерфейса сейчас чертом пишу чем надо делать и вот какая-то превьюшка что-то это ну не знаю я там буквально три вау вот и смотри вот файл и вот он рендерит,None,None,[],2024-09-18T17:38:08.180000Z,00:32,"(0, 32)",DG,0
3,3,Sergey Ryabenko,TBD,а что это за tx и и sx и sx что такое с,None,None,[],2024-09-18T17:38:17.358000Z,00:42,"(0, 42)",SR,1
4,4,,TBD,"Павел, такой...",None,None,[],2024-09-18T17:38:21.196000Z,00:46,"(0, 46)",NaN,1
5,5,Sergey Ryabenko,TBD,Вот этот?,Вот этот?,,[],2024-09-18T17:38:23.256000Z,00:48,"(0, 48)",SR,1
6,6,,TBD,Да.,Да.,,[],2024-09-18T17:38:23.936000Z,00:48,"(0, 48)",NaN,1
7,7,Dmitriy Grankin,TBD,Не знаю.,Не знаю.,,[],2024-09-18T17:38:25.056000Z,00:49,"(0, 49)",DG,1
8,8,Dmitriy Grankin,TBD,"Я хотел тебя спросить, что происходит.","Я хотел тебя спросить, что происходит.",,[],2024-09-18T17:38:27.636000Z,00:52,"(0, 52)",DG,1
9,9,Dmitriy Grankin,TBD,"Это React, да.","Это <b>React</b>, да.",React,[],2024-09-18T17:38:29.156000Z,00:53,"(0, 53)",DG,1


In [70]:
pd.options.display.max_colwidth = 1000
ideas_df = await MeetingExtraction.extract(formatted_input)
objects_df = await EntityExtraction.extract(formatted_input)
summary_df = pd.concat([ideas_df,objects_df.rename(columns={'entity': 'item'})]).reset_index(drop=True)
summary_refs = await SummaryIndexesRefs.extract(summary_df, formatted_input)

# Create a new dataframe for the references
ref_df = pd.DataFrame([(ref['summary_index'], r['s'], r['e']) 
                       for ref in summary_refs 
                       for r in ref['references']],
                      columns=['summary_index', 'start', 'end'])

# Merge the ref_df with entities_df
entities_with_refs = summary_df.reset_index().rename(columns={'index': 'summary_index'})
entities_with_refs = entities_with_refs.merge(ref_df, on='summary_index', how='left')

# Function to extract text from df based on start and end indices, including speaker
def get_text_range_with_speaker(row):
    text_range = df.loc[row['start']:row['end']]
    return ' | '.join(f"{speaker}: {content}" for speaker, content in zip(text_range['speaker'], text_range['content']))

# Apply the function to get the referenced text with speakers
entities_with_refs['referenced_text'] = entities_with_refs.apply(get_text_range_with_speaker, axis=1)

# Group by summary_index to combine multiple references
final_df = entities_with_refs.groupby('summary_index').agg({
    'item': 'first',
    'type': 'first',
    'summary': 'first',
    'details': 'first',
    'speaker': 'first',
    'referenced_text': ' | '.join
}).reset_index()

In [71]:
final_df[['item','type','summary','details','speaker']]

,item,type,summary,details,speaker
0,Current state of React project,discussion,"The meeting involved a discussion about the current state of a React project, focusing on the interface and backend integration.",Dmitriy Grankin and Sergey Ryabenko discussed the React files generated and their functionality. They noted that the project is in a working state but requires further exploration to understand its full capabilities and limitations.,Dmitriy Grankin
1,Layout generation in React,discussion,"The team explored how layout generation works in React, including the use of SX for styling.","Sergey Ryabenko explained that the React files generated include a layout version and that the process is legitimate. They acknowledged the need to delve deeper into the details of the implementation, especially regarding CSS.",Sergey Ryabenko
2,Understanding limitations,task,The team identified the need to understand the limitations of the current implementation to ensure effective future development.,"Dmitriy Grankin emphasized the importance of recognizing where the current project stands in terms of limitations, especially when compared to other MVPs being developed.",Dmitriy Grankin
3,Aligning with MVP standards,concern,The discussion highlighted the importance of aligning the project with MVP standards to ensure competitiveness.,"Sergey Ryabenko raised concerns about the project's approach to MVP development, suggesting that if the project diverges significantly from standard practices, it may face challenges in success.",Sergey Ryabenko
4,Dmitriy Grankin,person,"Dmitriy Grankin is a speaker in the meeting, involved in discussing the React project and its implementation details.","Dmitriy is curious about the React files and their functionality, asking questions about the project and expressing interest in understanding the backend processes.",Dmitriy Grankin
5,Sergey Ryabenko,person,"Sergey Ryabenko is a speaker in the meeting, providing insights into the React project and its components.","Sergey explains the generation of React files and discusses the layout and functionality of the project, contributing to the understanding of the implementation.",Sergey Ryabenko
6,React,topic,"React is a JavaScript library for building user interfaces, discussed in the context of a project in the meeting.","The meeting revolves around the use of React for generating files and creating a user interface, with both speakers exploring its capabilities and limitations.",Dmitriy Grankin
7,AppLayout.tsx,product,AppLayout.tsx is a specific file in the React project that was referenced during the meeting.,"Sergey asks to see the AppLayout.tsx file to understand its structure and how it fits into the overall project, indicating its importance in the layout generation.",Sergey Ryabenko
8,MVP,concept,"MVP stands for Minimum Viable Product, a concept discussed in relation to project development strategies.","Dmitriy mentions the importance of adhering to MVP standards in project development, emphasizing the need to keep up with others in the field.",Dmitriy Grankin
9,SX,topic,"SX refers to a styling solution used in the React project, discussed in the context of layout generation.","Sergey mentions SX in relation to the generated layout, indicating its role in the styling and presentation of the React components.",Sergey Ryabenko


In [72]:
from core import generic_call_stream

In [78]:
from pydantic import BaseModel, Field
from core import BaseCall
from typing import List

class Summary(BaseModel):
    summary: str = Field(..., description="Direct, to-the-point summary of the meeting")
    meeting_name: str = Field(..., description="Name of the meeting")

class MeetingSummary(BaseCall):
    output: Summary = Field(..., description="Meeting name and summary. Reference each item exactly as (item mentioned)[item index] .")

    @classmethod
    async def extract(cls, formatted_input: str, final_df: pd.DataFrame, model: str = "gpt-4o-mini"):
        output = await cls.call([
            system_msg("""Create a concise, direct summary incorporating all provided items from the data. 
                          Begin immediately with key points and content. Do not use any introductory phrases.
                          Absolutely avoid starting with phrases like "The meeting involved," "During the discussion," or any similar openings.
                          Reference each item exactly as (item mentioned)[item index] in the summary. 
                          Ensure all items are included and the summary flows naturally while capturing key points.
                          Use the full transcript for additional context if needed.
                          Avoid any temporal references or statements about the meeting itself."""),
            user_msg(f"""Here are the items and their details:

{final_df.reset_index()[['index','item', 'type', 'summary', 'details']].to_markdown()}

Full transcript:

{formatted_input}

Create a direct, to-the-point summary incorporating all these items. Start immediately with the content, referencing items exactly as (item mentioned)[item index]. Do not use any introductory phrases or mention that this is a summary of a meeting.""")
        ], model=model)

        return output[0].output

In [79]:
r = await MeetingSummary.extract(formatted_input, final_df)
print(r)

summary="The current state of the React project was discussed, focusing on interface and backend integration (Current state of React project)[0]. The team explored layout generation in React, including the use of SX for styling (Layout generation in React)[1]. There is a need to understand the limitations of the current implementation to ensure effective future development (Understanding limitations)[2]. Concerns were raised about aligning the project with MVP standards to ensure competitiveness (Aligning with MVP standards)[3]. Dmitriy Grankin participated in the discussion, expressing curiosity about the React files and their functionality (Dmitriy Grankin)[4]. Sergey Ryabenko provided insights into the React project, explaining the generation of React files and discussing layout and functionality (Sergey Ryabenko)[5]. React, a JavaScript library for building user interfaces, was central to the discussion (React)[6]. The AppLayout.tsx file was referenced as important for understandin

In [80]:
from IPython.display import Markdown
Markdown(r.summary)

The current state of the React project was discussed, focusing on interface and backend integration (Current state of React project)[0]. The team explored layout generation in React, including the use of SX for styling (Layout generation in React)[1]. There is a need to understand the limitations of the current implementation to ensure effective future development (Understanding limitations)[2]. Concerns were raised about aligning the project with MVP standards to ensure competitiveness (Aligning with MVP standards)[3]. Dmitriy Grankin participated in the discussion, expressing curiosity about the React files and their functionality (Dmitriy Grankin)[4]. Sergey Ryabenko provided insights into the React project, explaining the generation of React files and discussing layout and functionality (Sergey Ryabenko)[5]. React, a JavaScript library for building user interfaces, was central to the discussion (React)[6]. The AppLayout.tsx file was referenced as important for understanding the project's structure (AppLayout.tsx)[7]. The concept of MVP, or Minimum Viable Product, was emphasized as crucial for project development strategies (MVP)[8]. SX was mentioned as a styling solution used in the React project (SX)[9].

In [81]:
r.meeting_name

'React Project Discussion'

In [82]:
final_df

,summary_index,item,type,summary,details,speaker,referenced_text
0,0,Current state of React project,discussion,"The meeting involved a discussion about the current state of a React project, focusing on the interface and backend integration.",Dmitriy Grankin and Sergey Ryabenko discussed the React files generated and their functionality. They noted that the project is in a working state but requires further exploration to understand its full capabilities and limitations.,Dmitriy Grankin,"Dmitriy Grankin: показать и понять что тут вообще происходит вот я ему дал картинку интерфейса такой что картинку интерфейса сейчас чертом пишу чем надо делать и вот какая-то превьюшка что-то это ну не знаю я там буквально три вау вот и смотри вот файл и вот он рендерит | Sergey Ryabenko: а что это за tx и и sx и sx что такое с | : Павел, такой... | Sergey Ryabenko: Вот этот? | : Да. | Dmitriy Grankin: Не знаю. | Dmitriy Grankin: Я хотел тебя спросить, что происходит. | Dmitriy Grankin: Это React, да. | Sergey Ryabenko: А, это React файл, да-да-да. | Dmitriy Grankin: Да, я хотел с тобой посмотреть, мне просто интересно понять, что тут происходит и что с этим вообще можно делать. | Dmitriy Grankin: Как будто что-то даже работает. | Dmitriy Grankin: Ну, как бы да. Слушай, это он | Sergey Ryabenko: React генерит, и сразу можно превьюшку делать. | : Да, тут | : ну, дальше там | Dmitriy Grankin: с бэкэндом варится. Ну, там много, очень много видюшек в сети, как что дальш..."
1,1,Layout generation in React,discussion,"The team explored how layout generation works in React, including the use of SX for styling.","Sergey Ryabenko explained that the React files generated include a layout version and that the process is legitimate. They acknowledged the need to delve deeper into the details of the implementation, especially regarding CSS.",Sergey Ryabenko,"Sergey Ryabenko: Ну он как бы сгенерировал, да, эти прям реакторские файлы. | Sergey Ryabenko: Сразу их запустил. | Sergey Ryabenko: А можешь показать еще AppLayout.tsx? | Sergey Ryabenko: Ну, вот, который исходник. | Sergey Ryabenko: Ну, да. | Sergey Ryabenko: То есть он сразу и мог данные сгенерировал. | Sergey Ryabenko: Ну, слушай, ну, прикольно. | Sergey Ryabenko: Выглядит прям, да, живенько. | : Ладно, давай я | Dmitriy Grankin: поковыряю, потом посмотрим вместе. | Dmitriy Grankin: Давай. Ну, там же, видишь, дьявол кроется в деталях. | Sergey Ryabenko: когда до КСС доберешься, там прям какого-нибудь красивого, то все, пиши, пропало. | Dmitriy Grankin: Ну вот надо и понять, где в текущий момент находятся ограничения. | Dmitriy Grankin: я понял я смотрю ну как бы что-то тут происходит и над надо это использовать потому что ты будешь если Все остальные будут вот так вот делать MVP, а ты будешь делать как-то по-другому, то как бы нет шансов просто. | Sergey Ryabenko:..."
2,2,Understanding limitations,task,The team identified the need to understand the limitations of the current implementation to ensure effective future development.,"Dmitriy Grankin emphasized the importance of recognizing where the current project stands in terms of limitations, especially when compared to other MVPs being developed.",Dmitriy Grankin,"Dmitriy Grankin: Ну вот надо и понять, где в текущий момент находятся ограничения. | Dmitriy Grankin: я понял я смотрю ну как бы что-то тут происходит и над надо это использовать потому что ты будешь если Все остальные будут вот так вот делать MVP, а ты будешь делать как-то по-другому, то как бы нет шансов просто."
3,3,Aligning with MVP standards,concern,The discussion highlighted the importance of aligning the project with MVP standards to ensure competitiveness.,"Sergey Ryabenko raised concerns about the project's approach to MVP development, suggesting that if the project diverges significantly from standard practices, it may face challenges in success.",Sergey Ryabenko,"Dmitriy Grankin: я понял я смотрю ну как бы что-то тут происходит и над надо это использовать потому что ты будешь е

In [42]:
print(r.summary)


The meeting focused on evaluating candidates for a technical position, highlighting concerns about their qualifications and performance. Key points included: 1. Candidate Performance Evaluation: Nikita struggled with technical questions, raising doubts about his suitability, while another candidate was deemed overqualified, potentially leading to job dissatisfaction. 2. Importance of New Technologies: Dmitriy emphasized the need for developers to explore new technologies to remain competitive. 3. Coding Skills Assessment: A plan was proposed to assess a candidate's coding skills against a current developer, Andrew. 4. Need for Front-End Developers: The ongoing demand for skilled front-end developers was discussed, stressing the importance of finding candidates with both design and technical skills. Action items include evaluating the coding abilities of the new candidate and following up on front-end developer candidates.


In [34]:
r = await generic_call_stream([
    system_msg("""Create a concise, direct summary incorporating all provided items from the data. 
                  Start immediately with the key points, without any introductory phrases.
                  Reference each item exactly as [item name] in the summary. 
                  Ensure all items are included and the summary flows naturally while capturing key points.
                  Use the full transcript for additional context if needed.
                  Avoid any temporal references or introductory statements."""),
    user_msg(f"""Here are the items and their details:

{final_df[['item', 'type', 'summary', 'details']].to_markdown()}

Full transcript:

{formatted_input}

Create a direct, to-the-point summary incorporating all these items. Start immediately with the content, referencing items exactly as [item name]. Do not use any introductory phrases.


use markdown format, but a one nice readable text, not a list""")
])

Evaluated the performance of candidates for a technical position in [Candidate Performance Evaluation], noting inconsistencies in responses and qualifications. The first candidate, [Nikita], struggled to provide a coherent strategy for scaling to a million users, raising concerns about his understanding of the role. The second candidate was deemed overqualified, leading to [Overqualified Candidate Concerns] about job satisfaction and retention, as he expressed a desire to work with new technologies. [Importance of New Technologies] was emphasized by [Dmitriy Grankin], who highlighted the necessity for developers to explore new technologies to remain competitive. 

A [Coding Skills Assessment Plan] was proposed by [Sergey Ryabenko] to evaluate the new candidate's coding abilities against [Andrey]'s established performance. The ongoing need for front-end developers was discussed in [Need for Front-End Developers], with [Dmitriy] inquiring about the status of candidates who can effectivel

In [35]:
print(r)



Evaluated the performance of candidates for a technical position in [Candidate Performance Evaluation], noting inconsistencies in responses and qualifications. The first candidate, [Nikita], struggled to provide a coherent strategy for scaling to a million users, raising concerns about his understanding of the role. The second candidate was deemed overqualified, leading to [Overqualified Candidate Concerns] about job satisfaction and retention, as he expressed a desire to work with new technologies. [Importance of New Technologies] was emphasized by [Dmitriy Grankin], who highlighted the necessity for developers to explore new technologies to remain competitive. 

A [Coding Skills Assessment Plan] was proposed by [Sergey Ryabenko] to evaluate the new candidate's coding abilities against [Andrey]'s established performance. The ongoing need for front-end developers was discussed in [Need for Front-End Developers], with [Dmitriy] inquiring about the status of candidates who can effectivel

In [ ]:
docker run --name dima_entities -e POSTGRES_PASSWORD=mysecretpassword -p 5432:5432 -d postgres